In [2]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [3]:
%load_ext autoreload
%autoreload 2

# Define states and matrices

In [4]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

In [5]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.PICK_SMALLEST_PURVIEW = False

In [6]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([-1.+0.j, -0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [7]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [8]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PICK_SMALLEST_PURVIEW': False,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG'

In [9]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [10]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [11]:
# pyphi convention for state ordering is little endian
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [12]:
CNOT_tpm = pyphi.convert.to_2dimensional(pyphi.convert.state_by_state2state_by_node(CNOT_tpm.real))

In [13]:
network = pyphi.Network(CNOT_tpm)
subsystem = pyphi.Subsystem(network, (1,0))

In [14]:
ces = pyphi.compute.ces(subsystem)

In [13]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │    n0     ∅              ││    n0     ∅              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     ∅    n0              ││     ∅    n0              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

## Quantum results

In [15]:
rho = tensor(rho_plus, rho_zero)

In [16]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.5 0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.5 0. ]
 [0.  0.  0.  0. ]]

In [17]:
rho_e = evolve(rho, cnot(), 'effect')
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]

In [13]:
# Provide entanglement input, entangled -> ent_partition=[(0,1)]
# Not entangled, no input required
result = compute_ces(rho, cnot(), ent_partition=[(0,1)])
#result = compute_ces(rho, rho_e)

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  1.0
p:  (0, 1)  phi:  2.0


In [15]:
result

[{'mech': (0, 1),
  'purview': (0, 1),
  'phi': 2.0,
  'mip': 0,1    ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([-0.70710678-0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])]}]

In [65]:
# Diagonalize matrix
def diagonalize_matrix(rho):
    evals, evecs = rho.eigenstates()
    rho_D = rho.transform(evecs)
    return rho_D, evecs

In [84]:
rho_D, evecs = diagonalize_matrix(rho_e)

In [86]:
rho_D.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

In [87]:
evecs

array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.70710678]
        [ 0.        ]
        [ 0.        ]
        [-0.70710678]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.        ]
        [-0.70710678]
        [ 0.70710678]
        [ 0.        ]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[-0.70710678]
        [ 0.        ]
        [ 0.        ]
        [-0.70710678]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.        ]
        [-0.70710678]
        [-0.70710678]
        [ 0.        ]]                                                    ],
      dtype=object)

In [54]:
rho1 = evolve(tensor(rho_plus, rho_plus), cnot(), 'effect')
rho2 = evolve(tensor(rho_plus, rho_minus), cnot(), 'effect')
rho2

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25 -0.25 -0.25  0.25]
 [-0.25  0.25  0.25 -0.25]
 [-0.25  0.25  0.25 -0.25]
 [ 0.25 -0.25 -0.25  0.25]]

In [55]:
rho_e = rho1 + rho2
rho_e = rho_e.unit()
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.25 0.   0.   0.25]
 [0.   0.25 0.25 0.  ]
 [0.   0.25 0.25 0.  ]
 [0.25 0.   0.   0.25]]

In [58]:
0.5*(tensor(rho_plus, rho_plus) + tensor(rho_minus, rho_minus))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.25 0.   0.   0.25]
 [0.   0.25 0.25 0.  ]
 [0.   0.25 0.25 0.  ]
 [0.25 0.   0.   0.25]]

In [57]:
tensor(rho_minus, rho_minus)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25 -0.25 -0.25  0.25]
 [-0.25  0.25  0.25 -0.25]
 [-0.25  0.25  0.25 -0.25]
 [ 0.25 -0.25 -0.25  0.25]]

In [72]:
tensor(rho_minus, rho_minus).ptrace(1).purity()

0.9999999999999991

In [68]:
evacs

array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.        ]
        [ 0.        ]
        [-0.70710678]
        [ 0.70710678]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.        ]
        [ 0.81649658]
        [-0.40824829]
        [-0.40824829]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.8660254 ]
        [-0.28867513]
        [-0.28867513]
        [-0.28867513]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[0.5]
        [0.5]
        [0.5]
        [0.5]]                                                            ],
      dtype=object)